In [1]:
# Importing libraries
from pyedlut import simulation_wrapper as pyedlut
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
import os
import sys
import pandas as pd
import seaborn as sns

# Importing functions
sys.path.insert(1, '../data')
import figure_6_functions as fun
import simulation_function as sim


In [2]:
## Creating plots directory
RESULTS_DIR = "../results/figures_plots"
FIGURE_6_RESULTS_DIR = os.path.join(RESULTS_DIR, 'figure_6')
os.makedirs(FIGURE_6_RESULTS_DIR, exist_ok = True)

## Pattern Overlap Average

In [3]:
## Creating default parameters dictionary
my_dict_default = {"fraction":0, 
           "sigma": 40, 
            "seed": 0,
            "poisson_seed": 0,
            "noise": 0.0, 
            "seed_noise": 0,
            "mf_goc_w": 0.00,
            "grc_goc_w": 0.00,
            "num_patterns": 100,
            "duration_pattern": 0.08,
            "interval": 0.07,
            "firing_rate": 50.0, 
            "step": 1 
}

In [ ]:

## Creating dataframe
df = pd.DataFrame({'type':[], 
                   'mean':[],
                  'fraction':[],
                   'seed':[]})

seeds = [0, 1, 2, 3, 4, 5]

## Running simulations for 6 seeds
for s in seeds: 
    for n in range(10):

        ## creating dictionary that will iterate over the MF fraction and the seed
        my_dict = {
            **my_dict_default,
            "fraction": n, 
            "seed": s,
            "seed_noise": s,
        }

        mf_activity = fun.mf_activity(my_dict)
        oi, ot, _, matrix = sim.simulation(my_dict, mf_activity)

        # same dictionary with no inhibition
        my_dict_inh = {
            **my_dict_default,
            "fraction": n, 
            "seed": s,
            "seed_noise": s,
            "mf_goc_w": 0.0, 
            "grc_goc_w": 0.0,
        }

        mf_activity = fun.mf_activity(my_dict)
        oi, ot, _, matrix_inh = sim.simulation(my_dict, mf_activity)
        
        print(matrix.shape)
        
        ## Defining spikes count matrices for MFs, GrCs and GrCs with inhibition
        matrix_mf = matrix[:300,:]
        matrix_grc = matrix[300:4060,:]
        matrix_grc_inh = matrix_inh[300:4060,:]

        ########### MF #################################3
        binary_matrix_mf = matrix_mf/matrix_mf 
        binary_matrix_mf[np.isnan(binary_matrix_mf)] = 0
        overlap_matrix_mf = np.zeros((matrix_mf.shape[1], matrix_mf.shape[1]))

        ## Calculating overlap between active MFs for all the patterns
        for i in range(matrix_mf.shape[1]):
            for j in range(matrix_mf.shape[1]):
                num = np.where((binary_matrix_mf[:,i] == 1)&(binary_matrix_mf[:,j] == 1))[0].shape[0]
                overlap_matrix_mf[i,j] = num/matrix_mf.shape[0]

        ## Calculating overlap between active GrCs with no inhibition for all the patterns
        binary_matrix = matrix_grc/matrix_grc
        binary_matrix[np.isnan(binary_matrix)] = 0
        overlap_matrix_grc = np.zeros((matrix.shape[1], matrix_grc.shape[1]))

        for i in range(matrix_grc.shape[1]):
            for j in range(matrix_grc.shape[1]):
                num = np.where((binary_matrix[:,i] == 1)&(binary_matrix[:,j] == 1))[0].shape[0]
                overlap_matrix_grc[i,j] = num/matrix_grc.shape[0]


        mean_mf = np.mean(overlap_matrix_mf)
        mean_grc = np.mean(overlap_matrix_grc)

        ## Adding the overlap mean to the dataframe
        df.loc[len(df.index)] = ['MF', mean_mf, n*0.1 + 0.05, s] 
        df.loc[len(df.index)] = ['GRC (MF-GOC: ' + str(0.00) + ') (GRC-GOC: ' + str(round(0.00)) + ')', mean_grc, n*0.1 + 0.05, s] 

        # same dictionary increasing inhibition threshold
        my_dict_inh = {
            **my_dict_default,
            "fraction": n, 
            "seed": s,
            "seed_noise": s,
            "mf_goc_w": 0.1, 
            "grc_goc_w": 0.005,
        }

        
        mf_activity = fun.mf_activity(my_dict_inh)
        oi, ot, _, matrix_inh = sim.simulation(my_dict_inh, mf_activity)
        matrix_grc_inh = matrix_inh[300:4060,:]
        binary_matrix_grc_inh = matrix_grc_inh/matrix_grc_inh
        binary_matrix_grc_inh[np.isnan(binary_matrix_grc_inh)] = 0
        overlap_matrix_grc_inh = np.zeros((matrix_grc_inh.shape[1], matrix_grc_inh.shape[1]))

        for i in range(matrix_grc_inh.shape[1]):
            for j in range(matrix_grc_inh.shape[1]):
                num = np.where((binary_matrix_grc_inh[:,i] == 1)&(binary_matrix_grc_inh[:,j] == 1))[0].shape[0]
                overlap_matrix_grc_inh[i,j] = num/matrix_grc_inh.shape[0]

        ## Adding the overlap mean to the dataframe
        mean_grc_inh = np.mean(overlap_matrix_grc_inh)    
        df.loc[len(df.index)] = ['GRC (MF-GOC: ' + str(0.10) + ') (GRC-GOC: ' + str(round(0.005, 3)) + ')', mean_grc_inh, n*0.1 + 0.05, s] 

        # same dictionary increasing even more inhibition threshold
        my_dict_inh = {
            **my_dict_default,
            "fraction": n, 
            "seed": s,
            "seed_noise": s,
            "mf_goc_w": 0.1, 
            "grc_goc_w": 0.015,
        }

        mf_activity = fun.mf_activity(my_dict_inh)
        oi, ot, _, matrix_inh = sim.simulation(my_dict_inh, mf_activity)
        matrix_grc_inh = matrix_inh[300:4060,:]

        binary_matrix_grc_inh = matrix_grc_inh/matrix_grc_inh
        binary_matrix_grc_inh[np.isnan(binary_matrix_grc_inh)] = 0
        overlap_matrix_grc_inh = np.zeros((matrix_grc_inh.shape[1], matrix_grc_inh.shape[1]))

        for i in range(matrix_grc_inh.shape[1]):
            for j in range(matrix_grc_inh.shape[1]):
                num = np.where((binary_matrix_grc_inh[:,i] == 1)&(binary_matrix_grc_inh[:,j] == 1))[0].shape[0]
                overlap_matrix_grc_inh[i,j] = num/matrix_grc_inh.shape[0]

        ## Adding the overlap mean to the dataframe
        mean_grc_inh = np.mean(overlap_matrix_grc_inh)    
        df.loc[len(df.index)] = ['GRC (MF-GOC: ' + str(0.10) + ') (GRC-GOC: ' + str(round(0.015, 3)) + ')', mean_grc_inh, n*0.1 + 0.05, s]     

In [ ]:
## Setting colors
plt.rcParams['figure.figsize'] = (3,3)
colors = ['darkblue','green', 'orange'] # first color is black, last is red
cm = LinearSegmentedColormap.from_list(
        "Custom", colors, N=3)
mat = np.indices((1,3))[1]
plt.imshow(mat, cmap=cm)
plt.show()

###  Rearrenging dataframe...

In [ ]:
df['fraction'] = round(df['fraction'], 2)

In [ ]:
no_inh = df['type'] == 'GRC (MF-GOC: ' + str(0.00) + ') (GRC-GOC: ' + str(round(0.00)) + ')'
df_no_inh = df[no_inh].groupby('fraction')['mean'].agg(['mean', 'std']).reset_index()
df_no_inh['type'] = ['GrC (No Inh)']*df_no_inh.shape[0]

inh = df['type'] == 'GRC (MF-GOC: ' + str(0.10) + ') (GRC-GOC: ' + str(round(0.005, 3)) + ')'
df_inh = df[inh].groupby('fraction')['mean'].agg(['mean', 'std']).reset_index()
df_inh['type'] = ['GrC (+ Inh)']*df_inh.shape[0]

inh_plus = df['type'] == 'GRC (MF-GOC: ' + str(0.10) + ') (GRC-GOC: ' + str(round(0.015, 3)) + ')'
df_inh_plus = df[inh_plus].groupby('fraction')['mean'].agg(['mean', 'std']).reset_index()
df_inh_plus['type'] = ['GrC (++ Inh)']*df_inh_plus.shape[0]

mf = df['type'] == 'MF'
df_mf = df[mf].groupby('fraction')['mean'].agg(['mean', 'std']).reset_index()
df_mf['type'] = ['MF']*df_mf.shape[0]

df_final = pd.concat([df_mf, df_no_inh, df_inh, df_inh_plus], axis=0)

## OVERLAP BETWEEN PATTERNS PLOTS

In [ ]:

f_mf = np.linspace(0.05, 0.95, 10)
f_mf = [round(x, 2) for x in f_mf]
n_lines = 3
# cmap = mpl.colormaps['Greens']
cmap = cm 
# Take colors at regular intervals spanning the colormap.
#colors = cmap(np.linspace(0, 1, n_lines))


plt.rcParams['figure.figsize'] = (10,4)
plt.rcParams["savefig.facecolor"] = "white"
my_pal = ["red", "darkblue", "green", "orange"]
ax = sns.barplot(data = df_final, x = 'fraction', y = 'mean', hue = 'type', palette = my_pal, errorbar = "sd" )
ax.spines[['right', 'top']].set_visible(False)
ax.set_ylabel("Pattern overlap average", fontsize = 14)
ax.set_xlabel("MF active fraction", fontsize = 14)
ax.set_ylim([0.0, 1.0])

ax.tick_params(labelsize=14)
ax.legend(fontsize = 14)


In [ ]:
## Plotting pattern overlap average with standard deviations


f_mf = np.linspace(5, 95, 10)
f_mf = [int(x) for x in f_mf]

dfp = df_final.pivot(index='fraction', columns='type', values='mean')
dfp = dfp[['MF', 'GrC (No Inh)', 'GrC (+ Inh)', 'GrC (++ Inh)']]
# pivot the error
yerr = df_final.pivot(index='fraction', columns='type', values='std')
yerr = yerr[['MF', 'GrC (No Inh)', 'GrC (+ Inh)', 'GrC (++ Inh)']]
# plot
ax = dfp.plot(kind='bar', yerr=yerr, rot=0,  width=0.7, color=my_pal)

ax.spines[['right', 'top']].set_visible(False)
ax.set_ylabel("Pattern overlap average", fontsize = 14)
ax.set_xlabel("MF activation (%)", fontsize = 14)
ax.set_ylim([0.0, 1.0])
ax.tick_params(labelsize=14)
ax.legend(fontsize = 14)
ax.set_xticklabels(f_mf)

plt.savefig(os.path.join(FIGURE_6_RESULTS_DIR, 'overlap_average_all.png'), bbox_inches="tight", dpi = 200)